In [2]:
from dotenv import dotenv_values
import pandas as pd
from sqlalchemy import create_engine, text
from sshtunnel import SSHTunnelForwarder

In [3]:
# Load credentials from .env file
envDict = dotenv_values("../secrets/.env")

In [4]:
# Create a bind to forward connections on the local port to the mysql port on the server
server = SSHTunnelForwarder(
    # Host URL and Login
    envDict['SSH_HOST'],
    ssh_username = envDict['SSH_USERNAME'],
    # Private key for SSH connections
    ssh_pkey = envDict["SSH_PKEY_PATH"],
    ssh_private_key_password = envDict['SSH_PKEY_PASSWORD'],
    # Bind to mysql port on server
    remote_bind_address = (envDict["DB_REMOTE_REF_URL"], int(envDict["DB_REMOTE_REF_PORT"])),
    # Don't look for keys on the local machine
    allow_agent = False,
    host_pkey_directories = [],
)
# Start the tunneled connection
server.start()

In [5]:
# Create the engine to connect to the database using the tunnel
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(envDict["DB_USERNAME"], envDict["DB_PASSWORD"], "127.0.0.1", server.local_bind_port, "hospital_price_transparency"))

In [6]:
# This wildcard match crashes locally, takes about 35-40 seconds on the server
with engine.connect() as connection:
    prices_df = pd.read_sql_query(text("SELECT * FROM prices WHERE code LIKE '842%' LIMIT 1000000"), con=connection)
    hospital_df = pd.read_sql_query(text("SELECT * FROM hospitals WHERE npi_number in (SELECT npi_number FROM prices WHERE code LIKE '842%') LIMIT 1000000"), con=connection)

merged_df = pd.merge(prices_df, hospital_df, on='npi_number')

merged_df.head()

,code,npi_number,payer,price,name,url,street_address,city,state,zip_code,publish_date
0,842,1700979465,AETNA,14.94,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,None
1,842,1700979465,BC FL,15.69,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,None
2,842,1700979465,CHA HEALTH PLAN,14.94,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,None
3,842,1700979465,FL COMMUNITY CARE LTC,15.99,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,None
4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,None


In [7]:
print(len(merged_df)) #checking how many rows resulted from query

140835


In [8]:
uniques = merged_df["code"].unique()
print("Unique count: ", len(uniques))
print("Unique codes: ", uniques)

foo = merged_df.state.unique() #checking the variety of states
print(foo)

Unique count:  173
Unique codes:  ['842' '84202' '84202,1' '84203' '84203,1' '84206' '84206,1' '84207'
 '84207,1' '84210' '84210,1' '84220' '84220,1' '84228' '84228,1' '84233'
 '84233,1' '84234' '84234,1' '84235' '84235,1' '84238' '84238,1' '84244'
 '84244,1' '84252' '84252,1' '84255' '84255,1' '84260' '84260,1' '84270'
 '84270,1' '84275' '84275,1' '84285' '84285,1' '84295' '84295,1' '84210-1'
 '84234-1' '84244-1' '84295-1' '84295-00' '84295-01' '84202,2' '84202,3'
 '84206,2' '84206,3' '84206,4' '84207,2' '84207,3' '84210,2' '84220,2'
 '84220,3' '84220,4' '84220,5' '84231' '84238,2' '84238,3' '84238,4'
 '84238,5' '84238,6' '84244,2' '84244,3' '84244,4' '84252,2' '84255,2'
 '84255,3' '84255,4' '84260,2' '84260,3' '84260,4' '84260,5' '84260,6'
 '84260,7' '84270,2' '84270,3' '84270,4' '84275,2' '84275,3' '84295,2'
 '84295,3' '84295,4' '84295,9' '84207-00' '84207-01' '84234-00' '84234-01'
 '84220-00' '84220-01' '84275-00' '84275-01' '84202-00' '84202-01'
 '84202-02' '84202-03' '84270-00' '

In [93]:
with open('842entries.csv', 'w', encoding="utf-8") as f:
    f.write(merged_df.to_csv())